In [1]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
%%capture
!pip install transformers
!pip install datasets

In [3]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torch import optim
from torch.utils.data.dataset import Dataset, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

In [4]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = get_device()
print(device)

cuda


In [5]:
import pandas as pd

file_path ='/kaggle/input/vietnamese-sentiment-analyst/data.csv'

df = pd.read_csv(file_path)
df = df[['content', 'label']]

labels_map = {
    "POS": 0,
    "NEU": 1,
    "NEG": 2
}

df['label'] = df['label'].map(labels_map)
df = df.dropna(subset=['content'])

In [6]:
import re
import string

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F700-\U0001F77F"  
        u"\U0001F780-\U0001F7FF"  
        u"\U0001F800-\U0001F8FF"  
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA00-\U0001FA6F"  
        u"\U0001FA70-\U0001FAFF"  
        u"\U00002702-\U000027B0"  
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    text = text.lower() 
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'\d+', ' <num> ', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

In [7]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# import pandas as pd

# train_df = pd.read_pickle('/kaggle/input/sentiment-neu-neg/train_augmented.pkl')
# val_df = pd.read_pickle('/kaggle/input/sentiment-neu-neg/val.pkl')

In [9]:
train_df['content'] = train_df['content'].apply(remove_emoji)  
train_df['content'] = train_df['content'].apply(clean_text)  

val_df['content'] = val_df['content'].apply(remove_emoji)  
val_df['content'] = val_df['content'].apply(clean_text)  


In [10]:
train_df = train_df[['content', 'label']]
val_df = val_df[['content', 'label']]

In [11]:
from datasets import Dataset
from transformers import DataCollatorWithPadding, AutoTokenizer

# checkpoint = 'distilbert-base-multilingual-cased'
# tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding = True, max_length = 128)

2024-12-30 03:07:05.742460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 03:07:05.742598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 03:07:05.880773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [12]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25148 [00:00<?, ? examples/s]

Map:   0%|          | 0/6288 [00:00<?, ? examples/s]

In [13]:
train_dataset

Dataset({
    features: ['content', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 25148
})

In [14]:
# train_dataset = train_dataset.remove_columns(column_names='content')
# train_dataset = train_dataset.remove_columns(column_names='__index_level_0__')
# train_dataset = train_dataset.remove_columns(column_names='attention_mask')
# val_dataset = val_dataset.remove_columns(column_names='content')
# val_dataset = val_dataset.remove_columns(column_names='__index_level_0__')
# val_dataset = val_dataset.remove_columns(column_names='attention_mask')

In [15]:
train_dataset.set_format(type="torch", columns=["input_ids", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "label"])


In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, collate_fn=data_collator)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=10, collate_fn=data_collator)

In [17]:
# Lặp qua train_loader để xem từng batch
for batch in train_loader:
    # Lặp qua từng mẫu trong batch
    for key in batch.keys():
        print(f"Key: {key}, Shape: {batch[key].shape}")
    # Dừng lặp sau 1 batch để chỉ xem 1 lần
    break  # Bạn có thể loại bỏ break để lặp qua toàn bộ train

Key: input_ids, Shape: torch.Size([10, 69])
Key: attention_mask, Shape: torch.Size([10, 69])
Key: labels, Shape: torch.Size([10])


In [18]:
class MultiLayerBiLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, max_length, num_layers=2):
        super(MultiLayerBiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.num_layers = num_layers
        self.lstms = nn.ModuleList([
            nn.LSTM(input_size=embedding_dim if layer == 0 else hidden_size * 2,
                    hidden_size=hidden_size,
                    num_layers=1,
                    bidirectional=True,
                    batch_first=True)
            for layer in range(num_layers)
        ])
        self.dropout = nn.Dropout(0.3)
#         self.fc1 = nn.Linear(hidden_size *2, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
#         self.fc3 = nn.Linear(hidden_size // 2, output_size)
        
        self.fc1 = nn.Linear(hidden_size * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
    
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.to(torch.long)
        x = self.embedding(x)
        batch_size = x.size(0)
        
        for lstm in self.lstms:
            h0 = torch.zeros(2, batch_size, hidden_size).to(x.device)
            c0 = torch.zeros(2, batch_size, hidden_size).to(x.device)
            out, _ = lstm(x, (h0, c0))
            x = self.dropout(out)
        
        # Fully connected layers
        x = self.relu(self.fc1(x[:, -1, :]))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

In [19]:
# HYPER PARAMS
vocab_size = tokenizer.vocab_size
embedding_dim = 128
hidden_size = 128
output_size = 3
max_length = 128
num_epochs = 100
lr = 0.001
num_layers = 1
device = device
save_best_model_path = 'best_lstm_model.pt'
print(f"Vocabulary size: {vocab_size}")
print(f"Embedding dimension: {embedding_dim}")
print(f"Hidden size: {hidden_size}")
print(f"Output size: {output_size}")
print(f"Max length: {max_length}")
print(f"Number of epochs: {num_epochs}")

Vocabulary size: 36096
Embedding dimension: 128
Hidden size: 128
Output size: 3
Max length: 128
Number of epochs: 100


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb

def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=10, 
          save_best_model_path=None, save_last_model_path=None):
    
    best_val_loss = float('inf')
    best_val_accu = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        # Training loop
        for i, batch in enumerate(train_loader):
            inputs, labels = batch['input_ids'], batch['labels'] 
            
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_train_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
        
        # Calculate average train loss and accuracy for the epoch
        train_loss = running_train_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        
        # Log training metrics
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_accuracy})
        
        # Validation loop
        model.eval()
        running_val_loss = 0.0
        correct_val_predictions = 0
        total_val_predictions = 0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch['input_ids'], batch['labels']
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                val_loss = criterion(outputs, labels)
                running_val_loss += val_loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val_predictions += (predicted == labels).sum().item()
                total_val_predictions += labels.size(0)

        # Calculate average validation loss and accuracy
        val_loss = running_val_loss / len(val_loader)
        val_accuracy = correct_val_predictions / total_val_predictions
        
        # Log validation metrics
        wandb.log({"Val Loss": val_loss, "Val Accuracy": val_accuracy})
        
        # Print epoch statistics
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, \
              Val Loss: {val_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_accu = val_accuracy
            best_model_state_dict = model.state_dict()
            if save_best_model_path:
                torch.save(best_model_state_dict, save_best_model_path)
                print("Model saved")
        
    # Save last model
    if save_last_model_path:
        torch.save(model.state_dict(), save_last_model_path)
        
    print(f"Training finished. Best val accuracy is: {best_val_accu}")
    wandb.finish()  # Finish wandb session
    
    return best_model_state_dict, best_val_accu

# Evaluation function (for validation and testing)
def evaluate(model, loader, criterion, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    running_val_loss = 0.0
    
    with torch.no_grad():
        for i, batch in enumerate(loader):
            inputs, labels = batch['input_ids'], batch['labels']
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
    
    val_loss = running_val_loss / len(loader)
    val_accuracy = correct_predictions / total_predictions
    
    return val_loss, val_accuracy

# Example usage:
# Initialize your model, optimizer, criterion, and data loaders
# Then call train function passing all required parameters


In [ ]:
model = MultiLayerBiLSTMModel(vocab_size, embedding_dim, hidden_size, output_size, max_length, num_layers).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
wandb.init(
      project="lstm_sentiment_analysis", 
      name=f"wandb_example", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": lr,
      "architecture": "LSTM",
      "dataset": "vsa",
      "epochs": num_epochs,
      })
wandb.watch(model)
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=num_epochs)

wandb.finish()

wandb: Currently logged in as: vuminhhoang-vnu (vuminhhoang1512). Use `wandb login --relogin` to force relogin


Epoch [1/100], Train Loss: 0.9064,               Val Loss: 0.9142, Train Accuracy: 0.6412, Val Accuracy: 0.6279
Epoch [2/100], Train Loss: 0.8983,               Val Loss: 0.9152, Train Accuracy: 0.6414, Val Accuracy: 0.6279
Epoch [3/100], Train Loss: 0.8972,               Val Loss: 0.9156, Train Accuracy: 0.6416, Val Accuracy: 0.6277
